In [1]:
import pandas as pd
import numpy as np
import requests
import pyodbc
import json
# import colors
from urllib.error import HTTPError
from time import time, sleep
from watson_developer_cloud import ToneAnalyzerV3
from itertools import product
from werkzeug.datastructures import ImmutableMultiDict
import zipcodes
import pprint
pp = pprint.PrettyPrinter(indent=4)
from collections import defaultdict
import time

In [2]:
def sql_to_df(x):
    return pd.read_sql_query(x, db, index_col=None, coerce_float=True, params=None, parse_dates=None, chunksize=None)

def get_customers():
    query = "SELECT * FROM dbo.customer_basic"
    a = sql_to_df(query)
    return a

def debug(strSQL):
    print(strSQL)

In [3]:
server = 'tarpley.database.windows.net'
database = 'marketr'
username = 'tristan'
password = 'Fiverrtemp!'
driver= '{ODBC Driver 17 for SQL Server}'

connStr = 'DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password

db = pyodbc.connect(connStr)

In [4]:
def execute(query, returned, tup):

	cursor = db.cursor()
	retry_flag = True
	retry_count = 0
	while retry_flag and retry_count < 5:
		if retry_count == 4:
			return "Can you try again, please?"
		try:
			if returned == True:
				cursor.execute(query, tup)
				data = cursor.execute(query, tup)
				retry_flag = False
				return data, cursor
			else:
				cursor.execute(query, tup)
				cursor.close()
				retry_flag = False

		except:
			retry_count = retry_count + 1
			time.sleep(1)

In [40]:
df = pd.read_csv('audience.csv')

df = df.replace("'", '`', regex=True)
df = df.fillna('None')

df

,label,answer_id,page_id,order_on_page,score_weight_factor_0-10,Why_are_you_asking_binary,why_label,What_does_this mean_binary,what_label,Required_binary,prereq_tag_combo_id_binary,prereq_tag_combo_id_needed,label_p,horizontal_separator_binary
0,None,None,4,1,0,0,None,0,None,0,0,None,Think of a GREAT customer. Use them as a mode...,0
1,Let`s give this audience persona a name. Each ...,a0021,4,2,None,None,None,0,None,0,0,0,None,0
2,Gender (select all that apply),a0022,4,3,1,0,None,0,None,0,0,None,None,0
3,Target Age Groups (select all that apply),a0023,4,4,1,0,None,0,None,0,0,None,None,0
4,"In what city, state, US county, or country are...",a0024,4,5,1,0,None,0,None,0,0,None,None,0
5,How big is the company they work for?,a0025,4,6,1,0,None,0,None,0,2,tag0008\r\ntag0009\r\ntag0010\r\ntag0011\r\nta...,None,0
6,None,None,4,7,1,0,None,0,None,0,0,None,None,1
7,Time for the fun part! Quickly answer each qu...,None,4,8,1,0,None,0,None,0,0,None,None,0
8,FORMALITY,a0026,4,9,1,0,None,0,None,0,0,None,How formally would you speak with this person?,0
9,WHO ARE THEY BUYING THIS FOR?,a0027,4,10,1,0,None,0,None,0,0,None,For themselves or for their team or someone else?,0


In [28]:
base_query = """
INSERT INTO dbo.questions
(label, answer_id, page_id, order_on_page, score_weight_factor, why_asking_binary,why_label, what_binary, what_label, required_binary, prereq_tag_combo_id_binary, prereq_tag_combo_id_needed, label_p,horizontal_separator_binary)
VALUES
(?,?,?,?,?,?,?,?,?,?,?,?,?,?)
"""
for key, value in df.iterrows():
     
    tup = (value['label'], value['answer_id'], value['page_id'], value['order_on_page'], value['score_weight_factor_0-10'], value['Why_are_you_asking_binary'], value['why_label'], value['What_does_this mean_binary'], value['what_label'], value['Required_binary'], value['prereq_tag_combo_id_binary'], value['prereq_tag_combo_id_needed'], value['label_p'], value['horizontal_separator_binary'])
    
    
    
    print(tup)

(nan, nan, 4.0, 1.0, 0.0, 0.0, nan, 0.0, nan, 0.0, 0.0, nan, 'Think of a GREAT customer.  Use them as a model for this first persona and describe them by answering the following questions.  ', 0)
("Let's give this audience persona a name. Each of your audiences can have different names. Remember that this isn't supposed to be ALL of your customers—rather, a specific group of customers. You'll have a chance to add others at the end!", 'a0021', 4.0, 2.0, nan, nan, nan, 0.0, nan, 0.0, 0.0, '0', nan, 0)
('Gender (select all that apply)', 'a0022', 4.0, 3.0, 1.0, 0.0, nan, 0.0, nan, 0.0, 0.0, nan, nan, 0)
('Target Age Groups (select all that apply)', 'a0023', 4.0, 4.0, 1.0, 0.0, nan, 0.0, nan, 0.0, 0.0, nan, nan, 0)
('In what city, state, US county, or country are they mostly located?', 'a0024', 4.0, 5.0, 1.0, 0.0, nan, 0.0, nan, 0.0, 0.0, nan, nan, 0)
('How big is the company they work for?  ', 'a0025', 4.0, 6.0, 1.0, 0.0, nan, 0.0, nan, 0.0, 2.0, 'tag0008\r\ntag0009\r\ntag0010\r\ntag0011\r

In [203]:
def branch():
    action = 'hide'
    trigger = 'biz_model'
    biz_model = 'Affiliate'
    page = 'product'
    tup = (action, trigger, page)


    if action == 'hide':
        query = "SELECT ind, hide_val, branch_trigger_val FROM dbo.branches WHERE branch_action = ? AND branch_trigger=? AND affected_page=?"

        ind_query = "SELECT distinct ind FROM dbo.branches WHERE branch_action = 'hide' AND affected_page = ?"
        ind_list, cursor = execute(ind_query, True, (page,))
        ind_list = cursor.fetchall()
        cursor.close()

    elif action == 'mask':
        query = "SELECT mask_val, default_mask FROM dbo.branches WHERE branch_action = ? AND branch_trigger=? AND branch_trigger_val = ? AND affected_page=?"

    data, cursor = execute(query, True, tup)
    data = data.fetchall()

    if data:

        if action == 'hide':
            i = 0
            for ind in ind_list:
                ind_list[i] = ind_list[i][0]
                i+=1

            return ind_list, data

In [204]:
inds, hides = branch()
hides

[(1, 1, 'Affiliate'),
 (2, 1, 'Affiliate'),
 (8, 1, 'SaaS'),
 (9, 1, 'SaaS'),
 (11, 1, 'Digital Product'),
 (10, 1, 'Affiliate'),
 (10, 1, 'SaaS'),
 (10, 1, 'Professional Services'),
 (10, 1, 'Manual Services'),
 (10, 1, 'Media Provider'),
 (1, 1, 'Media Provider'),
 (2, 1, 'Media Provider'),
 (1, 1, 'Commission/Rev Share'),
 (2, 1, 'Commission/Rev Share'),
 (10, 1, 'Commission/Rev Share')]

In [208]:
inds

[1, 2, 8, 9, 10, 11]

In [200]:
biz_model = "Affiliate"
def dirty_mask_handler(biz_model, ind):
    for row in hides:
        if row[0] == ind:
            if row[2] == biz_model:
                return True
    return False

In [202]:
dirty_mask_handler(biz_model, 1)

True

[(1, 1), (2, 1), (10, 1)]

In [ ]:
hide_1 = dirty_mask_handler(inds[0], hide)
hide_2 = dirty_mask_handler(inds[1], hide)
hide_8 = dirty_mask_handler(inds[2], hide)
hide_9 = dirty_mask_handler(inds[3], hide)
hide_10 = dirty_mask_handler(inds[4], hide)
hide_11 = dirty_mask_handler(inds[5], hide)
hide_12 = dirty_mask_handler(inds[6], hide)

In [100]:
query = "SELECT distinct ind FROM dbo.branches WHERE branch_action = 'hide' AND affected_page = '%s'" % 'product'
cursor = db.cursor()
data = cursor.execute(query)
data = data.fetchall()

In [102]:
i = 0
for ind in data:
    data[i] = ind[0]
    i+=1
print(data)

[1, 2, 8, 9, 10, 11]


In [89]:
data

{1, 2, 8, 9, 10, 11}

In [88]:
data = set(data)
for val in data:
    print(val)

1
2
8
9
10
11


In [70]:
query = "SELECT hide_val, ind FROM dbo.branches WHERE branch_action = 'hide' AND branch_trigger='biz_model' AND branch_trigger_val = 'Professional Services' AND affected_page='product'"
cursor = db.cursor()
data = cursor.execute(query)
data = cursor.fetchall()

In [67]:
data

[(1, 10)]

In [31]:
if data:
    print(True)

True


In [69]:
class User:
    def __init__(self, id):
        self.id = id

    def set_biz_model(self):
        self.biz_model = get_biz_model(self.id)
    def set_selling_to(self):
        self.selling_to = get_selling_to(self.id)
    def set_num_products(self):
        self.num_products = get_num_products(self.id)

    def hide(self, page, index, trigger):
        self.set_biz_model()
        self.set_selling_to()
        self.set_num_products()
        cursor = db.cursor()
        if trigger == 'biz_model':
            query ="SELECT hide_val FROM dbo.branches WHERE branch_action = 'hide' AND branch_trigger=? AND branch_trigger_val = ? AND affected_page=? AND ind=?"
            test_query = query.replace("?", "%s")
            print(test_query % (trigger, self.biz_model, page, index))
            data = cursor.execute(query, (trigger, self.biz_model, page, index))
            print(self.biz_model)
        elif trigger == 'selling_to':
            query ="SELECT hide_val FROM dbo.branches WHERE branch_action = 'hide' AND branch_trigger=? AND branch_trigger_val = ? AND affected_page=? AND ind=?"
            test_query = query.replace("?", "%s")
            print(test_query % (trigger, self.selling_to, page, index))
            data = cursor.execute(query, (trigger, self.selling_to, page, index))
            

        data = cursor.fetchone()
        try:
            if data[0] == 1:
                cursor.close()
                return True
            else:
                cursor.close()
                return False
        except TypeError:
                print(data)
                return False

    def mask(self, page, index, trigger):
        self.set_biz_model()
        self.set_selling_to()
        self.set_num_products()
        cursor = db.cursor()
        if trigger == 'selling_to':
            data = cursor.execute("SELECT mask_val, default_mask FROM dbo.branches WHERE branch_action = 'mask' AND branch_trigger=? AND branch_trigger_val = ? AND affected_page=? AND ind=?", (trigger, self.selling_to, page, index))
        elif trigger == 'biz_model':
            data = cursor.execute("SELECT mask_val, default_mask FROM dbo.branches WHERE branch_action = 'mask' AND branch_trigger=? AND branch_trigger_val = ? AND affected_page=? AND ind=?", (trigger, self.selling_to, page, index))
        data = cursor.fetchone()
        try:
            mask = data[0]
            default = data[1]
            cursor.close()

            if mask != None:
                return mask, True
            else:
                return default, False
        except TypeError:
            new_data = cursor.execute("SELECT default_mask FROM dbo.branches WHERE branch_action = 'mask' AND branch_trigger=? AND affected_page=? AND ind=?", (trigger, page, index))
            new_data = cursor.fetchone()
            new_data = new_data[0]
            cursor.close()
            return new_data, False

In [70]:
me = User(115)
hide_10 = me.hide('product', 10, 'biz_model')
print(me.biz_model)

SELECT hide_val FROM dbo.branches WHERE branch_action = 'hide' AND branch_trigger=biz_model AND branch_trigger_val = biz_model AND affected_page=product AND ind=10
biz_model
None
biz_model


In [20]:
next_query = f"SELECT file_path FROM dbo.assets WHERE customer_id = 108"
data = cursor.execute(next_query)
data = cursor.fetchall()

for asset in data:
    print(asset[0])

static/uploads/img/Screen_Shot_2019-04-03_at_11.01.17_AM.png
static/uploads/img/Screen_Shot_2019-04-29_at_4.36.24_PM.png


In [58]:
def get_trigger_val(val, table, user):
    query = f"SELECT {val} FROM dbo.{table} WHERE customer_id = {user}"
    cursor = db.cursor()
    return_val = cursor.execute(query)
    return_val = cursor.fetchone()
    return_val = return_val[0]
    
    return return_val
    
def get_biz_model(user):
    x = get_trigger_val("biz_model", "company", 93)
    return x

def get_num_products(user):
    query = f"""SELECT COUNT(p_id)
                FROM dbo.product_list
                WHERE customer_id = {user}"""
    cursor = db.cursor()
    return_val = cursor.execute(query)
    return_val = cursor.fetchone()
    return_val = return_val[0]
    return return_val
    
def get_selling_to(user):
    x = get_trigger_val("selling_to", "company", 93)
    return x

In [9]:
load_company = load_company.drop(columns=['customer_id', 'age_group_1', 'age_group_2', 'age_group_3', 'age_group_4', 'age_group_5', 'age_group_6', 'age_group_7', 'age_group_8', 'before_1', 'before_2', 'before_3', 'before_4', 'before_5', 'before_6', 'before_7', 'before_8', 'before_9', 'before_10', 'before_freeform', 'after_1', 'after_2', 'after_3', 'after_4', 'after_5', 'after_6', 'after_7', 'after_8', 'after_9', 'after_10', 'after_freeform'])

In [10]:
load_company

,gender,location,why,audience_id,formality,buying_for,tech_savvy,decision_making,details,motive,persona_name
0,women,"Houston, Texas",Something just HAPPENED to them,41,Super Casual,For their team,Prefers simpler times,Ultimate decision maker,Both detailed oriented and big-picture,Save money,Test #1
1,male,"Dallas, Texas",Something they are PLANNING for,42,Super Casual,For their team,Prefers simpler times,Requires 1-2 others approval,Details matter most,Reduce risk,Test #2


In [110]:
def clean_audience():
    load_company = sql_to_df(f"""SELECT * FROM dbo.audience WHERE customer_id = 106""")
    ages_dict = {}
    ages = ['age_group_1','age_group_2','age_group_3','age_group_4','age_group_5','age_group_6','age_group_7','age_group_8']
    before_dict = {}
    before = ['before_1','before_2','before_3','before_4','before_5','before_6','before_7','before_8','before_9','before_10','before_freeform']
    after_dict = {}
    after = ['after_1','after_2','after_3','after_4','after_5','after_6','after_7','after_8','after_9','after_10', 'after_freeform']

    keys = ['ages', 'before', 'after']
    
    if len(load_company) > 1:
        t = 0
        temp = []

        while t<len(load_company):
            temp.append(t)
            t+=1

        i = 0 
        audiences_dict = dict.fromkeys(temp)

        x=0
        while x<len(load_company):
            audiences_dict[x] = {'ages':None, 'before':None, 'after':None}
            x += 1 



        while i < len(load_company):
            for key, value in load_company.iloc[i].iteritems():
                if value != '':
                    if key in ages:
                        ages_dict[key] = value
                    elif key in before:
                        before_dict[key] = value
                    elif key in after:
                        after_dict[key] = value


            audiences_dict[i]['ages'] = list(ages_dict.values())
            audiences_dict[i]['before'] = list(before_dict.values())
            audiences_dict[i]['after'] = list(after_dict.values())
            i += 1

            
    else:
        temp = [0]
        audiences_dict = dict.fromkeys(temp)
        for key, value in load_company.iloc[0].iteritems():
            if value != '':
                if key in ages:
                    ages_dict[key] = value
                elif key in before:
                    before_dict[key] = value
                elif key in after:
                    after_dict[key] = value
        audiences_dict[0] = {'after': after_dict, 'ages': ages_dict, 'before': before_dict}
            
    return audiences_dict

In [168]:
x = sql_to_df(f"""SELECT * FROM dbo.product_list WHERE customer_id = 106""")
for key, value in x.iterrows():
    for keys, value in x.iteritems():
        print("key: " + keys)
        print("value: " + str(value[key]))

key: complexity
value: None
key: price
value: None
key: product_or_service
value: None
key: frequency_of_use
value: None
key: frequency_of_purchase
value: None
key: value_prop
value: None
key: warranties_or_guarantee
value: None
key: warranty_guarantee_freeform
value: None
key: num_skus
value: None
key: level_of_customization
value: None
key: customer_id
value: 106
key: name
value: Big Jet
key: category
value: Jets
key: cogs
value: 7300
key: sales_price
value: 42000
key: qty_sold
value: 12
key: est_unique_buyers
value: 
key: p_id
value: 346
key: price_model
value: 
key: complexity
value: None
key: price
value: None
key: product_or_service
value: None
key: frequency_of_use
value: None
key: frequency_of_purchase
value: None
key: value_prop
value: None
key: warranties_or_guarantee
value: None
key: warranty_guarantee_freeform
value: None
key: num_skus
value: None
key: level_of_customization
value: None
key: customer_id
value: 106
key: name
value: Smaller Jet
key: category
value: Jets
key: 

In [119]:
def clean_product():
    load_company = sql_to_df(f"""SELECT * FROM dbo.product WHERE customer_id = 106""")
    
    segment_dict = {}
    segments = ['segment_1', 'segment_2', 'segment_3', 'segment_4', 'segment_5', 'segment_6', 'segment_7', 'segment_8', 'segment_9', 'segment_10']
    
    source_dict = {}
    sources = ['source_1','source_2','source_3','source_4', 'source_freeform']

    temp = [0]
    product_dict = dict.fromkeys(temp)
    for key, value in load_company.iloc[0].iteritems():
        if value != '':
            if key in segments:
                segment_dict[key] = value
            elif key in sources:
                source_dict[key] = value
        product_dict[0] = {'segment': list(segment_dict.values()), 'sources': list(source_dict.values())}
            
    return product_dict

In [44]:
if not x:
    print('hi!')
else:
    print('blah')

hi!


In [46]:
cursor = db.cursor()
data = cursor.execute("SELECT heading, paragraph FROM dbo.splash WHERE after_page = 'begin' ")
heading, paragraph = cursor.fetchone()

In [48]:
paragraph

'Progress will be saved as you go, so don`t worry about losing any progress as you begin completing your Marketing Plan.'

In [34]:
cursor.close()

In [303]:
branches = {}
indeces = {0: True}
pages = {"goals": indeces, "product": indeces, "product_2": indeces}

actions = {"hide": pages, "mask": pages}

biz_models = {"Affiliate":actions, "Digital Product": actions, "Manual Services": actions, "Media Provider": actions, "Professional Services": actions, "SaaS": actions, "Tangible Product": actions}

selling_to = {"B2B": biz_models, "B2C": biz_models, "C2C": biz_models, "other": biz_models}

triggers = {"num_products": selling_to, "selling_to": selling_to}

branches = {"trigger": triggers}

branches = defaultdict(branches)

TypeError: first argument must be callable or None

In [304]:
def create_branch(trigger, selling_to, biz_model, action, page, index, val):
    if action == "hide":
        branches["trigger"][trigger][selling_to][biz_model][action][page] = {index: val}
    elif action == "mask":
        print('hi')

In [318]:
trigger = "num_products"
selling_to = "B2B"
biz_model = "Affiliate"
action = "hide"
page = "product"
index = 1
val = False

In [325]:
def should_hide(page, index):
    cursor = db.cursor()
    data = cursor.execute(f"SELECT hide_val FROM dbo.branches WHERE branch_action = 'hide' AND branch_trigger_val = '{biz_model}' AND affected_page='{page}' AND ind={index}")
    data = cursor.fetchone()
    try:
        data = data[0]
        print(data)
        cursor.close()
        if data == 1:
            return True
        else:
            return False
    except:
        return False


In [347]:
cursor = db.cursor()
data = cursor.execute(f"SELECT hide_val, default_mask FROM dbo.branches WHERE branch_action = 'mask' AND branch_trigger_val = '{biz_model}' AND affected_page='{page}' AND ind={index}")
data = cursor.fetchone()
mask = data[0]
default = data[1]
cursor.close()
print(mask)

None


In [348]:
mask == None

True

In [342]:
should_hide(page, 1)

1


True

In [213]:
create_branch(trigger, selling_to, biz_model, action, page, index, val)

In [237]:
pp.pprint(branches["trigger"][trigger])

{   'B2B': {   'Affiliate': {   'hide': {   'goals': {0: True},
                                            'product': {0: True},
                                            'product_2': {0: True}},
                                'mask': {   'goals': {0: True},
                                            'product': {0: True},
                                            'product_2': {0: True}}},
               'Digital Product': {   'hide': {   'goals': {0: True},
                                                  'product': {0: True},
                                                  'product_2': {0: True}},
                                      'mask': {   'goals': {0: True},
                                                  'product': {0: True},
                                                  'product_2': {0: True}}},
               'Manual Services': {   'hide': {   'goals': {0: True},
                                                  'product': {0: True},
                        

In [9]:
class User:
    def __init__(self, id):
        self.id = id

    def set_biz_model(self):
        self.biz_model = get_biz_model(self.id)
    def set_selling_to(self):
        self.selling_to = get_selling_to(self.id)
    def set_num_products(self):
        self.num_products = get_num_products(self.id)

In [10]:
x = User(93)
x.set_biz_model()
x.set_num_products()
x.set_selling_to()
x.biz_model

'Digital Product'

In [12]:
user_lookup_table["biz_model"][x.biz_model][0]

'digital'

In [45]:
first_query = f"""INSERT INTO dbo.audience
                  (customer_id)
                  VALUES(93);commit;"""
cursor = db.cursor()

In [62]:
test = cursor.execute(f"SELECT TOP 2 audience_id FROM dbo.audience WHERE customer_id = 93 ORDER BY audience_id desc")
test = cursor.fetchall()
test[1][0]

31

In [44]:
names_and_ids = cursor.execute(f"SELECT persona_name, audience_id FROM dbo.audience WHERE customer_id = 94")
names_and_ids = cursor.fetchall()
names_and_ids[0][1]

IndexError: list index out of range

In [40]:
for name, id in names_and_ids:
    print(name)
    print(id)

Test #1
17
Test #2
18
Test #3
19
Test #4
29


In [22]:
return_val = return_val[0]
return_val

25

In [27]:
def clean_for_display(df):    
    for key, value in df.items():
        if key == "ID":
            df.rename(columns = {"ID": "Customer ID"}, inplace=True)
        elif key == "perc_complete":
            df.rename(columns = {"perc_complete": "% of onboarding completed"}, inplace=True)
        elif key.find("_") > 0: # if an underscore is found
            newkey = key.replace("_"," ") # convert underscores to space's
            df.rename(columns = {key: newkey}, inplace=True)
            
    return df

clean_for_display(test)
test

,is profile,Customer ID,first name,last name,email,password,company name,revenue,city,state,stage,employees,% of onboarding completed,last modified,account created,zip
0,True,19,Tristan,Tarpley,no,yes,Holdings Company,5132,Houston,FL,Shake-out,846531,30,2019-05-26 01:32:20.080,None,None


In [30]:
awareness = sql_to_df("select * from dbo.awareness WHERE customer_id=90")
awareness.insert(loc=0, column='stage', value='awareness')

evaluation = sql_to_df("select * from dbo.evaluation WHERE customer_id=90")
evaluation.insert(loc=0, column='stage', value='evaluation')

conversion = sql_to_df("select * from dbo.conversion WHERE customer_id=90")
conversion.insert(loc=0, column='stage', value='conversion')

retention = sql_to_df("select * from dbo.retention WHERE customer_id=90")
retention.insert(loc=0, column='stage', value='retention')

evaluation = sql_to_df("select * from dbo.evaluation WHERE customer_id=90")
evaluation.insert(loc=0, column='stage', value='evaluation')
stages = [awareness, evaluation, conversion, retention, evaluation]

result = pd.concat(stages)

In [31]:
result

,stage,uid,customer_id,tactic
0,awareness,14,90,test
1,awareness,15,90,testing1
0,evaluation,11,90,test
0,conversion,13,90,test
0,retention,14,90,test
0,evaluation,11,90,test


In [53]:
df = sql_to_df("""SELECT * FROM dbo.customer_basic WHERE id = '""" + str(19) + """'""")
df = clean_for_display(df)
df

,Customer ID,first name,last name,email,password,company name,revenue,city,state,stage,employees,% of onboarding completed,last modified,account created,zip
0,19,Tristan,Tarpley,no,yes,Holdings Company,5132,Houston,FL,Shake-out,846531,30,2019-05-21 20:59:45.950,None,None


In [25]:
first_query = sql_to_df("SELECT * FROM dbo.product_gen WHERE customer_id = '" + str(2) + "'")

#first_query['customer_id'][0] == None
first_query.head()

if first_query.empty == True:
    print('boo yah')

boo yah


In [7]:
query = """INSERT INTO dbo.product_list
    (name, complexity)
    VALUES ('test', 'test');commit;"""

In [8]:
d = {'key1': "yes", 'key2': "sir", 'key3': "i", 'key4': "am", 'key5': "im", 'key6': "home", 'key7': "asdf", 'key8': "asdf", 'key9': "asdf", 'key10': "asdf", 'key11': "asdf", 'key12': "asdf", 'key13': "wasd", 'key14': "wasd", 'key15': "wasd", 'key16': "wasd", 'key17': "wasd", 'key18': "wasd"}
z = 0
d1 = {}
for x, y in d.items():
    d1[str(z)] = y
    z = z+1

x = pd.DataFrame(list(d.items()))
test = list(x.iloc[:,1])
print(test)

['yes', 'sir', 'i', 'am', 'im', 'home', 'asdf', 'asdf', 'asdf', 'asdf', 'asdf', 'asdf', 'wasd', 'wasd', 'wasd', 'wasd', 'wasd', 'wasd']


In [9]:
q = 0
product_len = 3

while q < product_len:
    if q > 0:
        test = test[6:]
    
    query = """INSERT INTO dbo.product_list (customer_id,name,category,cogs,sales_price,qty_sold,est_unique_buyers) VALUES (0,'""" + test[0] + """','""" + test[1] + """','""" + test[2] + """','""" + test[3] + """','""" + test[4] + """','""" + test[5] + """')"""
    print(query)
    cursor = db.cursor()
    cursor.execute(query)
    cursor.close()
    q += 1

INSERT INTO dbo.product_list (customer_id,name,category,cogs,sales_price,qty_sold,est_unique_buyers) VALUES (0,'yes','sir','i','am','im','home')
INSERT INTO dbo.product_list (customer_id,name,category,cogs,sales_price,qty_sold,est_unique_buyers) VALUES (0,'asdf','asdf','asdf','asdf','asdf','asdf')
INSERT INTO dbo.product_list (customer_id,name,category,cogs,sales_price,qty_sold,est_unique_buyers) VALUES (0,'wasd','wasd','wasd','wasd','wasd','wasd')


In [10]:
df = sql_to_df('select * from dbo.product_list')

In [11]:
df.head(20)

,complexity,price,product_or_service,frequency_of_use,frequency_of_purchase,value_prop,warranties_or_guarantee,warranty_guarantee_freeform,num_skus,level_of_customization,customer_id,name,category,cogs,sales_price,qty_sold,est_unique_buyers,p_id
0,test,None,None,None,None,None,None,None,None,None,NaN,test,None,None,None,None,None,2
1,test,None,None,None,None,None,None,None,None,None,NaN,test,None,None,None,None,None,3
2,None,None,None,None,None,None,None,None,None,None,19.0,testing,test cat,8.35,9.24,98760,153879,11
3,None,None,None,None,None,None,None,None,None,None,19.0,testing123,2nd test,235.33,$8.14,098,098,12
4,None,None,None,None,None,None,None,None,None,None,19.0,testing,test cat,8.35,9.24,98760,153879,13
5,None,None,None,None,None,None,None,None,None,None,19.0,testing123,2nd test,235.33,$8.14,098,098,14
6,None,None,None,None,None,None,None,None,None,None,19.0,testing,test cat,8.35,9.24,98760,153879,15
7,None,None,None,None,None,None,None,None,None,None,19.0,testing123,2nd test,235.33,$8.14,098,098,16
8,None,None,None,None,None,None,None,None,None,None,19.0,testing,test cat,8.35,9.24,98760,153879,17
9,None,None,None,None,None,None,None,None,None,None,19.0,testing123,2nd test,235.33,$8.14,098,098,18


In [19]:
result = sql_to_df("SELECT * from dbo.customers WHERE email = 'test@test.com' AND password = 'test'")
if result['id'][0] != None:
    print(True)

True


In [20]:
query = """INSERT INTO dbo.customer_basic (
                                email,
                                password)
            VALUES ('""" + 'blah@blah.com' + """',
                    '""" + 'blah' + """'); commit;"""
    
cursor.execute(query)

In [28]:
cursor.close()

ProgrammingError: Attempt to use a closed cursor.

In [30]:
result = get_customers()

In [39]:
result.head()

,ID,first_name,last_name,facebook_ad_id,email,password
0,19,None,None,None,no,yes


In [44]:
POST_USERNAME = 'no'
POST_PASSWORD = 'yes'
result = sql_to_df("SELECT * from dbo.customer_basic WHERE email = '" + POST_USERNAME + "' AND password = '" + POST_PASSWORD + "'")

if result['ID'][0] != None:
    print(result['ID'][0])

19


In [45]:
result['ID'][0]

19

In [46]:
result.head()

,ID,first_name,last_name,facebook_ad_id,email,password,company_name,revenue,city,state,stage
0,19,test,test,None,no,yes,test,651320,test,AL,Shake-out


In [6]:
vision_subscription_key = '8359f1b134914ab1a1137c5a1d9d547c'
assert vision_subscription_key

text_subscription_key = '45f31e1a295e414b8d71319b79a405e0'
assert text_subscription_key

vision_base_url = "https://southcentralus.api.cognitive.microsoft.com/vision/v1.0/"
vision_analyze_url = vision_base_url + "analyze"
image_url = 'https://images.wagwalkingweb.com/media/breed/dachshund/appearance/dachshund.png?auto=compress&fit=max'

vision_headers  = {'Ocp-Apim-Subscription-Key': vision_subscription_key }
vision_params   = {'visualFeatures': 'Categories,Description,Color'}

In [7]:
def img_attr_to_memory():
    image_attr_query = "select * from dbo.image_attributes as dboi"
    image_attr_df = sql_to_df(image_attr_query)
    image_attr_df = image_attr_df.drop_duplicates(subset=['image_hash'], keep='last')
    
    return image_attr_df

In [8]:
image_attr_df = img_attr_to_memory()

In [9]:
def get_img_attr(img, row_select):
    vision_data     = {'url': img}
    try:
        vision_response = requests.post(vision_analyze_url, headers=vision_headers, params=vision_params, json=vision_data)
        vision_response.raise_for_status()

        analysis = vision_response.json()

        accent_color = analysis['color']['accentColor']
        dom_color_back = analysis['color']['dominantColorBackground']
        dom_color_fore = analysis['color']['dominantColorForeground']

        is_bw = analysis['color']['isBwImg']

        height = analysis['metadata']['height']
        width = analysis['metadata']['width']  

        try:
            confidence = analysis['description']['captions'][0]['confidence']
            caption = analysis['description']['captions'][0]['text']
            tags_1 = analysis['description']['tags'][0]
            dom_colors_1 = analysis['color']['dominantColors'][0]

            update_statement = """UPDATE dbo.image_attributes 
                                    SET accent_color = '""" + str(accent_color) + """',
                                    dominant_color_background = '""" + dom_color_back + """',
                                    dominant_color_foreground = '""" + dom_color_fore + """',
                                    dominant_color_1 = '""" + dom_colors_1 + """',
                                    is_bw_img = '""" + str(is_bw) + """',
                                    tags_1 = '""" + tags_1 + """',
                                    caption = '""" + caption + """',
                                    confidence = """ + str(confidence) + """,
                                    height = """ + str(height) + """,
                                    width = """ + str(width) + """
                                    WHERE id='""" + row_select + """';
                                  commit;"""

        except IndexError:
            try:
                #without tags
                confidence = analysis['description']['captions'][0]['confidence']
                caption = analysis['description']['captions'][0]['text']

                dom_colors_1 = analysis['color']['dominantColors'][0]

                update_statement = """UPDATE dbo.image_attributes 
                                        SET accent_color = '""" + str(accent_color) + """',
                                            dominant_color_background = '""" + dom_color_back + """',
                                            dominant_color_foreground = '""" + dom_color_fore + """',
                                            dominant_color_1 = '""" + dom_colors_1 + """',
                                            is_bw_img = '""" + str(is_bw) + """',
                                            caption = '""" + caption + """',
                                            confidence = """ + str(confidence) + """,
                                            height = """ + str(height) + """,
                                            width = """ + str(width) + """
                                            WHERE id='""" + row_select + """';
                                      commit;"""

            except IndexError:
                try:
                    #without caption, add tags
                    confidence = analysis['description']['captions'][0]['confidence']
                    tags_1 = analysis['description']['tags'][0]     
                    dom_colors_1 = analysis['color']['dominantColors'][0]

                    update_statement = """UPDATE dbo.image_attributes 
                                            SET accent_color = '""" + str(accent_color) + """',
                                                dominant_color_background = '""" + dom_color_back + """',
                                                dominant_color_foreground = '""" + dom_color_fore + """',
                                                dominant_color_1 = '""" + dom_colors_1 + """',
                                                is_bw_img = '""" + str(is_bw) + """',
                                                tags_1 = '""" + tags_1 + """',
                                                confidence = """ + str(confidence) + """,
                                                height = """ + str(height) + """,
                                                width = """ + str(width) + """
                                                WHERE id='""" + row_select + """';
                                          commit;"""

                except IndexError:
                    try:
                        #without caption and without tags 
                        dom_colors_1 = analysis['color']['dominantColors'][0]
                        confidence = analysis['description']['captions'][0]['confidence']
                        update_statement = """UPDATE dbo.image_attributes 
                                                SET accent_color = '""" + str(accent_color) + """',
                                                    dominant_color_background = '""" + dom_color_back + """',
                                                    dominant_color_foreground = '""" + dom_color_fore + """',
                                                    dominant_color_1 = '""" + dom_colors_1 + """',
                                                    is_bw_img = '""" + str(is_bw) + """',
                                                    confidence = """ + str(confidence) + """,
                                                    height = """ + str(height) + """,
                                                    width = """ + str(width) + """
                                                    WHERE id='""" + row_select + """';
                                              commit;"""

                    except IndexError:
                        try:
                            #without caption, tags, and dom_colors_1
                            confidence = analysis['description']['captions'][0]['confidence']

                            update_statement = """UPDATE dbo.image_attributes 
                                                    SET accent_color = '""" + str(accent_color) + """',
                                                        dominant_color_background = '""" + dom_color_back + """',
                                                        dominant_color_foreground = '""" + dom_color_fore + """',
                                                        is_bw_img = '""" + str(is_bw) + """',
                                                        confidence = """ + str(confidence) + """,
                                                        height = """ + str(height) + """,
                                                        width = """ + str(width) + """
                                                        WHERE id='""" + row_select + """';
                                                commit;"""
                        except IndexError:
                            try:
                                #without confidence, tags, caption, and dom_colors
                                update_statement = """UPDATE dbo.image_attributes 
                                                        SET accent_color = '""" + str(accent_color) + """',
                                                            dominant_color_background = '""" + dom_color_back + """',
                                                            dominant_color_foreground = '""" + dom_color_fore + """',
                                                            is_bw_img = '""" + str(is_bw) + """',
                                                            height = """ + str(height) + """,
                                                            width = """ + str(width) + """
                                                            WHERE id='""" + row_select + """';
                                                      commit;"""
                            except IndexError:
                                try:
                                    #same as above, add confidence and caption
                                    caption = analysis['description']['captions'][0]['text']
                                    confidence = analysis['description']['captions'][0]['confidence']
                                    update_statement = """UPDATE dbo.image_attributes 
                                                            SET accent_color = '""" + str(accent_color) + """', 
                                                            dominant_color_background = '""" + dom_color_back + """', 
                                                            dominant_color_foreground = '""" + dom_color_fore + """', 
                                                            is_bw_img = '""" + str(is_bw) + """', 
                                                            caption = '""" + caption + """', 
                                                            confidence = """ + str(confidence) + """, 
                                                            height = """ + str(height) + """, 
                                                            width = """ + str(width) + """ 
                                                            WHERE id='""" + row_select + """';
                                                          commit;"""
                                except IndexError:
                                    print('idk')

    except requests.exceptions.HTTPError as err:
        print(err)
        return
                    
    return update_statement


In [10]:
def update_img_attr(df):
    counter = 0
    for index, row in df.iterrows(): 
        starttime = time()
        counter += 1
        if counter < 20:
            update_statement = get_img_attr(row['image_url'], row['id'])

            print(index)
            debug(update_statement)
            if update_statement == None:
                continue
            else:
                cursor.execute(update_statement)    
        else:
            counter = 0
            sleep(60.0 - ((time() - starttime) % 60.0))
        
        continue

In [11]:
text_attr_query = "select * from dbo.text_attributes as dboi"
text_attr_df = sql_to_df(text_attr_query)
text_attr_df = text_attr_df.dropna(subset=["body"], how='any')
text_attr_df = text_attr_df.loc[660:]

In [16]:
text_analytics_base_url = "https://southcentralus.api.cognitive.microsoft.com/text/analytics/v2.0/"
language_api_url = text_analytics_base_url + "languages"
sentiment_api_url = text_analytics_base_url + "sentiment"
key_phrase_api_url = text_analytics_base_url + "keyPhrases"
tone_analyzer = ToneAnalyzerV3(
    version ='2017-09-21',
    username ='e4a9a55c-764e-4799-8c00-b0886bb6f019',
    password ='bMhV1vmUBzaq'
)

In [13]:
def get_emotional(text, row_select):
    content_type = 'application/json'
    res = tone_analyzer.tone({"text": text},content_type)
    pos = 0
    
    try:

        while pos < (len(res['sentences_tone']) - 1):
            pos += 1
            i = 0
            if res['sentences_tone'][pos]['tones'] != []:
                while i <= (len(res['sentences_tone'][pos]['tones']) - 1):

                    score = res['sentences_tone'][pos]['tones'][i]['score']
                    tone_name = res['sentences_tone'][pos]['tones'][i]['tone_name']    
                    sample_text = res['sentences_tone'][pos]['text']

                    update_statement = """UPDATE dbo.text_attributes
                                            SET tone_""" + str(pos) + """ = '""" + str(tone_name) + """',
                                                text_sample_""" + str(pos) + """ = '""" + str(sample_text) + """',
                                                score_""" + str(pos) + """ = """ + str(score) + """
                                          WHERE id='""" + str(row_select) + """'; commit;""" 

                    debug(update_statement)
                    try:
                        cursor.execute(update_statement)
                    except:
                        break

                    i += 1
            else:
                continue



    except KeyError:
        while pos <= (len(res['document_tone']['tones']) - 1):
            score = res['document_tone']['tones'][pos]['score']
            tone_name = res['document_tone']['tones'][pos]['tone_name']


            pos += 1
            
            update_statement = """UPDATE dbo.text_attributes
                                    SET tone_""" + str(pos) + """ = '""" + str(tone_name) + """',
                                        score_""" + str(pos) + """ = """ + str(score) + """
                                  WHERE id='""" + str(row_select) + """'; commit;""" 
            
            debug(update_statement)
            try:
                cursor.execute(update_statement)
            except:
                break

In [14]:
def update_emotions():
    for index, row in text_attr_df.iterrows():
        starttime = time()

        print(index)
        body_copy = row['body'].replace("'", "")
        get_emotional(body_copy, row['id'])

In [17]:
def get_key_phrases(text, row_select, err):
    text_doc = {'documents' : [
        { 'id': '1', 'language': 'en', 'text': text },
    ]}
    
    key_phrases_headers   = {"Ocp-Apim-Subscription-Key": text_subscription_key}
    key_phrases_response  = requests.post(key_phrase_api_url, headers=key_phrases_headers, json=text_doc)
    key_phrases = key_phrases_response.json()
    phrase_len = len(key_phrases['documents'][0])
    
    try:
        if phrase_len == 1:
            key_1 = key_phrases['documents'][0]['keyPhrases'][0]
            
            update_statement = "UPDATE dbo.text_attributes SET key_phrases_1 = '" + str(key_1) + "' WHERE id='" + row_select + "'; commit;"
            
        elif phrase_len == 2:
            key_1 = key_phrases['documents'][0]['keyPhrases'][0]
            key_2 = key_phrases['documents'][0]['keyPhrases'][1]
            
            update_statement = "UPDATE dbo.text_attributes SET key_phrases_1 = '" + str(key_1) + "', key_phrases_2 = '" + str(key_2) + "' WHERE id='" + row_select + "'; commit;"
            
            
        elif phrase_len == 3:
            key_1 = key_phrases['documents'][0]['keyPhrases'][0]
            key_2 = key_phrases['documents'][0]['keyPhrases'][1]
            key_3 = key_phrases['documents'][0]['keyPhrases'][2]
            
            update_statement = "UPDATE dbo.text_attributes SET key_phrases_1 = '" + str(key_1) + "', key_phrases_2 = '" + str(key_2) + "', key_phrases_2 = '" + str(key_3) + "' WHERE id='" + row_select + "'; commit;"
            
        elif phrase_len == 4:
            key_1 = key_phrases['documents'][0]['keyPhrases'][0]
            key_2 = key_phrases['documents'][0]['keyPhrases'][1]
            key_3 = key_phrases['documents'][0]['keyPhrases'][2]
            key_4 = key_phrases['documents'][0]['keyPhrases'][3]
            
            update_statement = "UPDATE dbo.text_attributes SET key_phrases_1 = '" + str(key_1) + "', key_phrases_2 = '" + str(key_2) + "', key_phrases_2 = '" + str(key_3) + "', key_phrases_4 = '" + str(key_4) + "' WHERE id='" + row_select + "'; commit;"
            
        elif phrase_len == 5:
            key_1 = key_phrases['documents'][0]['keyPhrases'][0]
            key_2 = key_phrases['documents'][0]['keyPhrases'][1]
            key_3 = key_phrases['documents'][0]['keyPhrases'][2]
            key_4 = key_phrases['documents'][0]['keyPhrases'][3]
            key_5 = key_phrases['documents'][0]['keyPhrases'][4]
            
            update_statement = "UPDATE dbo.text_attributes SET key_phrases_1 = '" + str(key_1) + "', key_phrases_2 = '" + str(key_2) + "', key_phrases_2 = '" + str(key_3) + "', key_phrases_4 = '" + str(key_4) + "', key_phrases_5 = '" + str(key_5) + "' WHERE id='" + row_select + "'; commit;"
        
        else:
            return
        
    except IndexError:
        print(key_phrases)
        print(err)
        update_statement = str(key_phrases) + " " + str(err)
    except KeyError:
        print(key_phrases)
        print(err)
        update_statement = str(key_phrases) + " " + str(err)
        
    return update_statement

In [18]:
def update_key_phrases_attr():
    counter = 0
    for index, row in text_attr_df.iterrows():
        starttime = time()
        counter += 1
        if counter < 20:
            update_statement = get_key_phrases(row['body'], row['id'], index)
            print(index)
            debug(update_statement)

            if update_statement == None:
                continue
            else:
                try:
                    cursor.execute(update_statement)
                except:
                    continue

        else:
            counter = 0
            sleep(60.0 - ((time() - starttime) % 60.0))

In [19]:
def get_sentiment_attr(text, row_select, err):
    text_doc = {'documents' : [
        { 'id': '1', 'language': 'en', 'text': text },
    ]}
    a = json.dumps(text_doc)
    
    sentiment_headers   = {"Ocp-Apim-Subscription-Key": text_subscription_key}
    sentiment_response  = requests.post(sentiment_api_url, headers=sentiment_headers, json=text_doc)
    sentiments = sentiment_response.json()
    
    try:
        sentiment_score = sentiments['documents'][0]['score']    
        update_statement = "UPDATE dbo.text_attributes SET sentiment = " + str(sentiment_score) + " WHERE id='" + row_select + "'; commit;"
        
    except IndexError:
        print(sentiments)
        print(err)
        update_statement = str(sentiments) + " " + str(err)
    except KeyError:
        print(sentiments)
        print(err)
        update_statement = str(sentiments) + " " + str(err)
    
    return update_statement

In [20]:
def update_text_attr():
    counter = 0
    for index, row in text_attr_df.iterrows():
        starttime = time()
        counter += 1
        if counter < 20:
            update_statement = get_sentiment_attr(row['body'], row['id'], index)
            print(index)
            debug(update_statement)
            
            if update_statement == None:
                continue
            else:
                cursor.execute(update_statement)
        
        else:
            counter = 0
            sleep(60.0 - ((time() - starttime) % 60.0))

In [23]:
actions_query = """
                    SELECT  ad_id,
                            clicks,
                            cost_per_10_sec_video_view_value,
                            cpc,
                            cpp,
                            cpm,
                            ctr,
                            impressions,
                            reach,
                            relevance_score_score,
                            relevance_score_status
                            
                            
                    FROM    dbo.ad_insights
                """
actions_df = sql_to_df(actions_query)

In [24]:
actions_df['cpa'] = actions_df[['cost_per_10_sec_video_view_value', 'cpc', 'cpp', 'cpm']].mean(axis=1)

In [25]:
attr_query = """SELECT ia.*, ta.*, body, image_url, ads.id as ad_id
                FROM   dbo.image_attributes as ia,
                       dbo.text_attributes as ta,
                       dbo.ads as ads
                WHERE  ta.creative_id = ia.creative_id
                   AND ads.creative_id = ta.creative_id
                   AND ads.creative_id = ia.creative_id"""
attr_df = sql_to_df(attr_query)

In [26]:
actions_attr_df = pd.merge(actions_df, attr_df, on='ad_id')
actions_attr_df.drop_duplicates(subset=['ad_id'], keep='last', inplace=True)
actions_attr_df.sort_values(by='cpa', inplace=True)
actions_attr_df['confidence'].fillna(actions_attr_df.confidence.mean(), inplace=True)

# Colors

In [27]:
def hex_to_hsv(h):
    r, g, b = (tuple(int(h[i:i+2], 16) for i in (0, 2 ,4)))
    h, s, v = colors.rgb_to_hsv(r, g, b)
    
    return h*360, s*100, v/255

In [28]:
def color_name(h):
    
    h, s, v = hex_to_hsv(h)
    
    if h < 60:
        h = 'red'
    elif h >= 60 and h < 120:
        h = 'yellow'
    elif h >= 120 and h < 180:
        h = 'green'
    elif h >= 180 and h < 240:
        h = 'cyan'
    elif h >= 240 and h < 300:
        h = 'blue'
    elif h >= 300 and h <= 360:
        h = 'magenta'
        
    return h

In [29]:
def color_saturation(h): 
    h, s, v = hex_to_hsv(h)
    
    if s < 25:
        s = 'dim saturation'
    elif s >= 25 and s < 50:
        s = 'low saturation'
    elif s >= 50 and s < 75:
        s = 'moderate saturation'
    elif s >= 75 and s <= 100:
        s = 'high saturation'
        
    return s

In [30]:
def color_brightness(h):
    h, s, v = hex_to_hsv(h)
    
    if v < 25:
        v = 'dim brightness'
    elif v >= 25 and v < 50:
        v = 'low brightness'
    elif v >= 50 and v < 75:
        v = 'moderate brightness'
    elif v >= 75 and v <= 100:
        v = 'high brightness'
    
    return v

In [32]:
actions_attr_df['accent_color_name'] = actions_attr_df['accent_color'].apply(color_name)
actions_attr_df['color_saturation'] = actions_attr_df['accent_color'].apply(color_saturation)
actions_attr_df['color_brightness'] = actions_attr_df['accent_color'].apply(color_brightness)
actions_attr_df['accent_color'] = actions_attr_df['accent_color'].fillna(actions_attr_df['accent_color'].mode()[0])

In [33]:
def back_analysis(df):
    df.sort_values(by='cpa', inplace=True)
    df = df[df['relevance_score_status'] == 'OK']
    df = df[0:4]

    new_df = df[['sentiment', 'key_phrases_1', 'tone_1', 'tone_2', 'accent_color_name', 'color_saturation', 'color_brightness', 'dominant_color_background', 'body', 'image_url']].copy()

In [34]:
color_var_list = ['accent_color_name', 'color_saturation', 'color_brightness', 'dominant_color_background', 'dominant_color_foreground', 'dominant_color_1']
text_var_list = ['sentiment', 'tone_1']

In [36]:
color_variables = pd.DataFrame(actions_attr_df.groupby(color_var_list).cpa.mean())
color_variables.reset_index(level=color_var_list, inplace=True)
color_variables.set_index('cpa', inplace=True)
color_variables.sort_values(by='cpa', inplace=True)

In [38]:
text_variables = pd.DataFrame(actions_attr_df.groupby(text_var_list).cpa.mean())
text_variables.reset_index(level=text_var_list, inplace=True)
text_variables.set_index('cpa', inplace=True)
text_variables.sort_values(by='cpa', inplace=True)

In [39]:
text_and_color = pd.merge(text_variables, color_variables, on='cpa')

def get_top_cpas():
    b = text_and_color
    cpa_trends = b.index.values

    return cpa_trends

In [40]:
def avg_cpa(df, var, attr):
    df = df[df[var] == attr]
    indices = df.index.values
    index_avg = indices.mean()
    return index_avg

In [41]:
def cpa_lookup(df):
    rows, columns = df.shape
    i = 0
    lookup_table = {}
    col_val = df.columns.values
    while i < columns:
        col_sel = df.iloc[:, i]
        for row in col_sel:
            col_header = col_val[i]
            avg = avg_cpa(df, col_header, row)
            lookup_table[row] = avg
            #print("attr: " + str(row) + "\n avg_cpa: $" + str(avg) + "\n\n")
        i += 1
    return lookup_table

In [42]:
def find_in_lookup(val):
    data = cpa_lookup(text_and_color)
    
    for key, value in data.items():
        if key == val:
            return value

In [43]:
def est_cpa(df, row_set):
    rows, columns = df.shape
    i = 0
    temp = {}
    row_sel = df.iloc[row_set, :]

    for val in row_sel:
        if i < 7:
            temp[i] = find_in_lookup(val)
            i+=1
        else:
            break
                      
    mean = float(sum(temp.values())) / len(temp)
    return mean

In [44]:
def petri_dish():
    accent_color_name = text_and_color['accent_color_name']
    color_saturation = text_and_color['color_saturation']
    color_brightness = text_and_color['color_brightness']
    dominant_color_background = text_and_color['dominant_color_background']
    dominant_color_foreground = text_and_color['dominant_color_foreground']
    dominant_color_1 = text_and_color['dominant_color_1']
    sentiment = text_and_color['sentiment']
    tone_1 = text_and_color['tone_1']
    test_combos = list(product(accent_color_name, color_saturation, color_brightness, dominant_color_background, dominant_color_foreground, dominant_color_1, tone_1))
    blah = pd.DataFrame(test_combos)
    
    rows, columns = blah.shape
    blah = blah.drop_duplicates(subset=[0,1,2,3,4,5,6], keep='first')
#     blah = blah[:10]
    blah_indeces = blah.index.values
    blah['est_cpa'] = ""
    
    def update_cpa():
        i=0
        for row in blah_indeces:
            blah['est_cpa'][row] = est_cpa(blah, i)
            i += 1
            
    update_cpa()

    blah.sort_values(by='est_cpa', inplace=True)
    #blah.drop_duplicates(subset=['est_cpa'], keep='last', inplace=True)
    
    return blah